In [1]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob

In [9]:
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'rBxKg9uEhYFUpXniAAlYLwdrD'
        consumer_secret = 'rz95QqXA0ZB6RbzSG43iafc8bRyWalXhotDSPujURezbyRNObW'
        access_token = '1457558916-UULZe4Fm146BBpLX3j5qWXV3kcRVS9vxfoVjGHY'
        access_token_secret = 'b3tc3bVFeoJL7h11vEULy877CAo6ETloB88sLAiOHhAiH'
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed")
            
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split())
                               
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e))

In [25]:
def main(): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'Kashmir', count = 200) 
  
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    print(len(ntweets))
    print(len(ptweets))
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
    print("Neutral tweets percentage: {} % \ ".format(100*(len(tweets) - abs(len(ntweets) - len(ptweets)))/len(tweets))) 
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text']) 
  
if __name__ == "__main__": 
    # calling main function 
    main() 

Positive tweets percentage: 23.076923076923077 %
16
21
Negative tweets percentage: 17.582417582417584 %
Neutral tweets percentage: 94.50549450549451 % \ 


Positive tweets:
RT @anita_chauhan80: Pic of the Day.💞

Love &amp; peace in our valley. #Kashmir 

#JaiHind 🇮🇳🇮🇳

@SwetaSinghAT @GautamGambhir @ImRaina @gauravc…
RT @AartiTikoo: Some activists want to see people dying in Kashmir; it makes fascinating optics, great news stories &amp; titiliating politics…
RT @arunv2808: Mumbai's population is almost as equal to that of Australia. Trains run at atleast 30x capacity
Roads are filled with pothol…
RT @madhukishwar: With friends like you, Kashmiris need no enemies. How much time have you spent in Kashmir? Most Kashmiris, including thos…
"Freedom is the power to choose our own chains"
FREE KASHMIR
#EidForKashmiriSacrifices
Rory is right to raise concerns about the situation in Kashmir. I recognise they are two sides to this such as Paki… https://t.co/NWY9lgXFCS
RT @TIinExile: Only Muslim M